In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization, LSTM, GRU
from sklearn.utils import resample
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [2]:
# Load the dataset
df = pd.read_csv('combodataset.csv', header = None)

df.columns = ['age', 'sex', 'cp', 'trestbps', 'chol',
              'fbs', 'restecg', 'thalach', 'exang', 
              'oldpeak', 'slope', 'target']

df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,target
0,40,1,2,140,289,0,0,172,0,0.0,1,0
1,49,0,3,160,180,0,0,156,0,1.0,2,1
2,37,1,2,130,283,0,1,98,0,0.0,1,0
3,48,0,4,138,214,0,0,108,1,1.5,2,1
4,54,1,3,150,195,0,0,122,0,0.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1185,45,1,1,110,264,0,0,132,0,1.2,2,1
1186,68,1,4,144,193,1,0,141,0,3.4,2,1
1187,57,1,4,130,131,0,0,115,1,1.2,2,1
1188,57,0,2,130,236,0,2,174,0,0.0,2,1


In [3]:
# Separate features and target variable
X = df.drop('target', axis=1)
y = df['target']


In [4]:

# 1. Load and preprocess the data
# Assuming you have your data in the X and y variables

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)



In [5]:
input_shape = (X_train.shape[1], 1)
input_shape_cnn = (X_train.shape[1], 1)

In [6]:
# 4. Hyperparameter tuning
learning_rate = 0.001
batch_size = 50
epochs = 100



# 5. Learning rate scheduling
lr_scheduler = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=learning_rate, decay_steps=10000, decay_rate=0.9
)
optimizer = keras.optimizers.Adam(learning_rate=lr_scheduler)



# 6. Regularization techniques
callback_early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)
callback_checkpoint = keras.callbacks.ModelCheckpoint(
    'model_checkpoint.h5', save_best_only=True
)



In [7]:
def build_cnn_model(input_shape):
    model = Sequential()
    model.add(Conv1D(filters=11, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.25))
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))  # Output layer
    
    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model


In [10]:
# Train the individual models
cnn_model = build_cnn_model(input_shape)
cnn_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test),
             callbacks=[callback_early_stopping, callback_checkpoint])


Epoch 1/100
20/20 [==============================] - 2s 24ms/step - loss: 0.5746 - accuracy: 0.7468 - val_loss: 0.5272 - val_accuracy: 0.7563
Epoch 2/100
20/20 [==============================] - 0s 9ms/step - loss: 0.5211 - accuracy: 0.7616 - val_loss: 0.5403 - val_accuracy: 0.6597
Epoch 3/100
20/20 [==============================] - 0s 9ms/step - loss: 0.4601 - accuracy: 0.7815 - val_loss: 0.4283 - val_accuracy: 0.7941
Epoch 4/100
20/20 [==============================] - 0s 9ms/step - loss: 0.4310 - accuracy: 0.8130 - val_loss: 0.4413 - val_accuracy: 0.7731
Epoch 5/100
20/20 [==============================] - 0s 10ms/step - loss: 0.4267 - accuracy: 0.8078 - val_loss: 0.4566 - val_accuracy: 0.7689
Epoch 6/100
20/20 [==============================] - 0s 10ms/step - loss: 0.4220 - accuracy: 0.8183 - val_loss: 0.4046 - val_accuracy: 0.7983
Epoch 7/100
20/20 [==============================] - 0s 9ms/step - loss: 0.4316 - accuracy: 0.8099 - val_loss: 0.4087 - val_accuracy: 0.7983
Epoch 8/10

In [11]:
# 8. Evaluate the model
test_loss, test_accuracy = cnn_model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')


8/8 [==============================] - 0s 3ms/step - loss: 0.3339 - accuracy: 0.8571
Test Loss: 0.3339
Test Accuracy: 0.8571


In [12]:
# Make predictions
y_pred = cnn_model.predict(X_test)
y_pred_classes = np.round(y_pred)


# Calculate confusion matrix and classification report
from sklearn.metrics import confusion_matrix, classification_report


conf_matrix = confusion_matrix(y_test, y_pred_classes)
class_report = classification_report(y_test, y_pred_classes)

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)


8/8 [==============================] - 0s 3ms/step
Confusion Matrix:
[[ 94  13]
 [ 21 110]]

Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.88      0.85       107
           1       0.89      0.84      0.87       131

    accuracy                           0.86       238
   macro avg       0.86      0.86      0.86       238
weighted avg       0.86      0.86      0.86       238



In [22]:
# 4. Hyperparameter tuning
learning_rate = 0.001
batch_size = 50
epochs = 100



# 5. Learning rate scheduling
lr_scheduler = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=learning_rate, decay_steps=10000, decay_rate=0.9
)
optimizer = keras.optimizers.Adam(learning_rate=lr_scheduler)



# 6. Regularization techniques
callback_early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)
callback_checkpoint = keras.callbacks.ModelCheckpoint(
    'model_checkpoint.h5', save_best_only=True
)



In [23]:
def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(11, return_sequences=True, input_shape=input_shape))
    model.add(LSTM(32, return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(LSTM(64, return_sequences=True))
    model.add(LSTM(32))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Output layer
    
    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model


In [24]:
lstm_model = build_lstm_model(input_shape)
lstm_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test),
              callbacks=[callback_early_stopping, callback_checkpoint])


Epoch 1/100
20/20 [==============================] - 15s 176ms/step - loss: 0.6171 - accuracy: 0.6891 - val_loss: 0.6410 - val_accuracy: 0.7101
Epoch 2/100
20/20 [==============================] - 1s 35ms/step - loss: 0.5696 - accuracy: 0.7080 - val_loss: 0.6583 - val_accuracy: 0.7101
Epoch 3/100
20/20 [==============================] - 1s 36ms/step - loss: 0.5645 - accuracy: 0.7080 - val_loss: 0.6619 - val_accuracy: 0.6555
Epoch 4/100
20/20 [==============================] - 1s 34ms/step - loss: 0.5645 - accuracy: 0.7269 - val_loss: 0.6752 - val_accuracy: 0.5504
Epoch 5/100
20/20 [==============================] - 1s 34ms/step - loss: 0.5616 - accuracy: 0.7227 - val_loss: 0.6572 - val_accuracy: 0.6975
Epoch 6/100
20/20 [==============================] - 1s 35ms/step - loss: 0.5511 - accuracy: 0.7269 - val_loss: 0.6624 - val_accuracy: 0.6597
Epoch 7/100
20/20 [==============================] - 1s 35ms/step - loss: 0.5435 - accuracy: 0.7269 - val_loss: 0.6578 - val_accuracy: 0.6134
Epoc

20/20 [==============================] - 1s 35ms/step - loss: 0.2695 - accuracy: 0.8855 - val_loss: 0.3649 - val_accuracy: 0.8613
Epoch 59/100
20/20 [==============================] - 1s 35ms/step - loss: 0.2565 - accuracy: 0.8950 - val_loss: 0.3127 - val_accuracy: 0.8992
Epoch 60/100
20/20 [==============================] - 1s 43ms/step - loss: 0.2411 - accuracy: 0.9002 - val_loss: 0.2998 - val_accuracy: 0.9034
Epoch 61/100
20/20 [==============================] - 1s 41ms/step - loss: 0.2401 - accuracy: 0.9044 - val_loss: 0.2934 - val_accuracy: 0.8866
Epoch 62/100
20/20 [==============================] - 1s 40ms/step - loss: 0.2356 - accuracy: 0.9013 - val_loss: 0.2925 - val_accuracy: 0.8992
Epoch 63/100
20/20 [==============================] - 1s 34ms/step - loss: 0.2243 - accuracy: 0.9065 - val_loss: 0.3187 - val_accuracy: 0.8992
Epoch 64/100
20/20 [==============================] - 1s 35ms/step - loss: 0.2199 - accuracy: 0.9044 - val_loss: 0.3386 - val_accuracy: 0.8739
Epoch 65/100

In [25]:
# 8. Evaluate the model
test_loss, test_accuracy = lstm_model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')


8/8 [==============================] - 0s 10ms/step - loss: 0.2811 - accuracy: 0.8992
Test Loss: 0.2811
Test Accuracy: 0.8992


In [26]:
# Make predictions
y_pred = lstm_model.predict(X_test)
y_pred_classes = np.round(y_pred)


# Calculate confusion matrix and classification report
from sklearn.metrics import confusion_matrix, classification_report


conf_matrix = confusion_matrix(y_test, y_pred_classes)
class_report = classification_report(y_test, y_pred_classes)

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)


8/8 [==============================] - 2s 9ms/step
Confusion Matrix:
[[ 92  15]
 [  9 122]]

Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.86      0.88       107
           1       0.89      0.93      0.91       131

    accuracy                           0.90       238
   macro avg       0.90      0.90      0.90       238
weighted avg       0.90      0.90      0.90       238



In [38]:
# 4. Hyperparameter tuning
learning_rate = 0.001
batch_size = 50
epochs = 100



# 5. Learning rate scheduling
lr_scheduler = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=learning_rate, decay_steps=10000, decay_rate=0.9
)
optimizer = keras.optimizers.Adam(learning_rate=lr_scheduler)



# 6. Regularization techniques
callback_early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)
callback_checkpoint = keras.callbacks.ModelCheckpoint(
    'model_checkpoint.h5', save_best_only=True
)



In [39]:
def build_gru_model(input_shape):
    model = Sequential()
    model.add(GRU(11, return_sequences=True, input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(GRU(32, return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(GRU(64, return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(GRU(128, return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(GRU(128))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Output layer
    
    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model


In [40]:
gru_model = build_gru_model(input_shape)
gru_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test),
             callbacks=[callback_early_stopping, callback_checkpoint])


Epoch 1/100
20/20 [==============================] - 21s 225ms/step - loss: 0.6581 - accuracy: 0.6817 - val_loss: 0.6912 - val_accuracy: 0.5504
Epoch 2/100
20/20 [==============================] - 1s 66ms/step - loss: 0.4894 - accuracy: 0.7763 - val_loss: 0.6954 - val_accuracy: 0.5504
Epoch 3/100
20/20 [==============================] - 1s 74ms/step - loss: 0.4737 - accuracy: 0.7658 - val_loss: 0.6897 - val_accuracy: 0.5504
Epoch 4/100
20/20 [==============================] - 1s 65ms/step - loss: 0.4543 - accuracy: 0.7994 - val_loss: 0.6906 - val_accuracy: 0.5504
Epoch 5/100
20/20 [==============================] - 1s 73ms/step - loss: 0.4651 - accuracy: 0.7952 - val_loss: 0.6767 - val_accuracy: 0.5504
Epoch 6/100
20/20 [==============================] - 1s 74ms/step - loss: 0.4451 - accuracy: 0.8015 - val_loss: 0.6431 - val_accuracy: 0.5798
Epoch 7/100
20/20 [==============================] - 1s 74ms/step - loss: 0.4506 - accuracy: 0.7994 - val_loss: 0.6320 - val_accuracy: 0.5966
Epoc

20/20 [==============================] - 1s 64ms/step - loss: 0.3968 - accuracy: 0.8246 - val_loss: 0.5241 - val_accuracy: 0.7605
Epoch 59/100
20/20 [==============================] - 1s 65ms/step - loss: 0.4069 - accuracy: 0.8277 - val_loss: 0.4812 - val_accuracy: 0.8151
Epoch 60/100
20/20 [==============================] - 1s 68ms/step - loss: 0.4155 - accuracy: 0.8162 - val_loss: 0.4384 - val_accuracy: 0.8277
Epoch 61/100
20/20 [==============================] - 1s 68ms/step - loss: 0.4101 - accuracy: 0.8172 - val_loss: 0.3761 - val_accuracy: 0.8361
Epoch 62/100
20/20 [==============================] - 2s 92ms/step - loss: 0.4106 - accuracy: 0.8151 - val_loss: 0.3603 - val_accuracy: 0.8529
Epoch 63/100
20/20 [==============================] - 1s 70ms/step - loss: 0.4082 - accuracy: 0.8162 - val_loss: 0.3466 - val_accuracy: 0.8445


In [41]:
# 8. Evaluate the model
test_loss, test_accuracy = gru_model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')


8/8 [==============================] - 0s 13ms/step - loss: 0.3120 - accuracy: 0.8613
Test Loss: 0.3120
Test Accuracy: 0.8613


In [42]:
# Make predictions
y_pred = gru_model.predict(X_test)
y_pred_classes = np.round(y_pred)


# Calculate confusion matrix and classification report
from sklearn.metrics import confusion_matrix, classification_report


conf_matrix = confusion_matrix(y_test, y_pred_classes)
class_report = classification_report(y_test, y_pred_classes)

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)


8/8 [==============================] - 2s 14ms/step
Confusion Matrix:
[[ 88  19]
 [ 14 117]]

Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.82      0.84       107
           1       0.86      0.89      0.88       131

    accuracy                           0.86       238
   macro avg       0.86      0.86      0.86       238
weighted avg       0.86      0.86      0.86       238



In [43]:
# Make predictions using individual models
cnn_predictions = (cnn_model.predict(X_test) > 0.5).astype("int32")
lstm_predictions = (lstm_model.predict(X_test) > 0.5).astype("int32")
gru_predictions = (gru_model.predict(X_test) > 0.5).astype("int32")



8/8 [==============================] - 0s 14ms/step


In [44]:
# Combine predictions using a majority voting ensemble
ensemble_predictions = np.mean([cnn_predictions, lstm_predictions, gru_predictions], axis=0) >= 0.5
ensemble_predictions = ensemble_predictions.astype("int32")

In [45]:
# Calculate accuracy of the ensemble
ensemble_accuracy = accuracy_score(y_test, ensemble_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")

Ensemble Accuracy: 0.8908


In [46]:
# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, ensemble_predictions)

print("Confusion Matrix:")
print(conf_matrix)

# Print classification report
class_report = classification_report(y_test, ensemble_predictions)
print("Classification Report:")
print(class_report)

# Evaluate ensemble accuracy
ensemble_accuracy = accuracy_score(y_test, ensemble_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")


Confusion Matrix:
[[ 94  13]
 [ 13 118]]
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.88      0.88       107
           1       0.90      0.90      0.90       131

    accuracy                           0.89       238
   macro avg       0.89      0.89      0.89       238
weighted avg       0.89      0.89      0.89       238

Ensemble Accuracy: 0.8908


In [47]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.utils import resample
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from sklearn.model_selection import cross_val_score

In [48]:
# List of base models

lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train, y_train)

svm_l_model = SVC(kernel="linear")
svm_l_model.fit(X_train, y_train)

svm_p_model = SVC(kernel="poly")
svm_p_model.fit(X_train, y_train)

svm_r_model = SVC(kernel="rbf")
svm_r_model.fit(X_train, y_train)

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)


knn_model = KNeighborsClassifier()
knn_model.fit(X_train, y_train)

gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb_model.fit(X_train, y_train)

ab_model = AdaBoostClassifier(n_estimators=100, random_state=42)
ab_model.fit(X_train, y_train)

xgb_model = XGBClassifier(n_estimators=100, random_state=42)
xgb_model.fit(X_train, y_train)

gpc_model = GaussianProcessClassifier(random_state=42)
gpc_model.fit(X_train, y_train)

GaussianProcessClassifier(random_state=42)

In [49]:
lr_predictions = lr_model.predict(X_test)
svm_l_predictions = svm_l_model.predict(X_test)
svm_p_predictions = svm_p_model.predict(X_test)
svm_r_predictions = svm_r_model.predict(X_test)
rf_predictions = rf_model.predict(X_test)
knn_predictions = knn_model.predict(X_test)
gb_predictions = gb_model.predict(X_test)
ab_predictions = ab_model.predict(X_test)
xgb_predictions = xgb_model.predict(X_test)
gpc_predictions = gpc_model.predict(X_test)


In [50]:
# Combine predictions using ensemble (Voting Classifier)
ensemble_predictions = np.column_stack((cnn_predictions, lstm_predictions, gru_predictions, lr_predictions, 
                                        svm_l_predictions, svm_p_predictions, svm_r_predictions, 
                                        rf_predictions, knn_predictions, gb_predictions, 
                                        ab_predictions, xgb_predictions, gpc_predictions))


In [51]:
# Perform majority voting to get the final prediction
final_predictions = np.mean(ensemble_predictions, axis=1) >= 0.5
final_predictions = final_predictions.astype("int32")

In [52]:
# Calculate accuracy of the ensemble
ensemble_accuracy = accuracy_score(y_test, final_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")

Ensemble Accuracy: 0.9076


In [53]:
# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, final_predictions)

print("Confusion Matrix:")
print(conf_matrix)

# Print classification report
class_report = classification_report(y_test, final_predictions)
print("Classification Report:")
print(class_report)

# Evaluate ensemble accuracy
ensemble_accuracy = accuracy_score(y_test, final_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")

Confusion Matrix:
[[ 92  15]
 [  7 124]]
Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.86      0.89       107
           1       0.89      0.95      0.92       131

    accuracy                           0.91       238
   macro avg       0.91      0.90      0.91       238
weighted avg       0.91      0.91      0.91       238

Ensemble Accuracy: 0.9076


In [54]:
# Combine predictions using ensemble (Voting Classifier)
ensemble_predictions = np.column_stack((cnn_predictions, lstm_predictions, gru_predictions, svm_r_predictions, 
                                        rf_predictions, knn_predictions, gb_predictions, 
                                        xgb_predictions, gpc_predictions))


In [55]:
# Perform majority voting to get the final prediction
final_predictions = np.mean(ensemble_predictions, axis=1) >= 0.5
final_predictions = final_predictions.astype("int32")

# Calculate accuracy of the ensemble
ensemble_accuracy = accuracy_score(y_test, final_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, final_predictions)

print("Confusion Matrix:")
print(conf_matrix)

# Print classification report
class_report = classification_report(y_test, final_predictions)
print("Classification Report:")
print(class_report)

# Evaluate ensemble accuracy
ensemble_accuracy = accuracy_score(y_test, final_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")

Ensemble Accuracy: 0.9118
Confusion Matrix:
[[ 92  15]
 [  6 125]]
Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.86      0.90       107
           1       0.89      0.95      0.92       131

    accuracy                           0.91       238
   macro avg       0.92      0.91      0.91       238
weighted avg       0.91      0.91      0.91       238

Ensemble Accuracy: 0.9118
